In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
train_data = pd.read_excel('train_data.xlsx')

# Preprocessing

## Handle missing values

In [4]:
print("Handling Missing Values...")
train_data = train_data.dropna(thresh=0.5 * len(train_data), axis=1)  # Drop columns with >50% missing

numerical_cols = ['cibil_score', 'total_no_of_acc', 'annual_inc', 'int_rate', 
                  'loan_amnt', 'installment', 'account_bal', 'emp_length']
categorical_cols = ['sub_grade', 'term', 'home_ownership', 'purpose', 
                    'application_type', 'verification_status']

Handling Missing Values...


## Impute numerical and categorical values

In [5]:
for col in numerical_cols:
    train_data[col] = train_data[col].fillna(train_data[col].median())

In [6]:
for col in categorical_cols:
    train_data[col] = train_data[col].fillna(train_data[col].mode()[0])

In [7]:
print("Missing values handled.")

Missing values handled.


## Treating the Outliers

In [8]:
for col in numerical_cols:
    upper_limit = train_data[col].quantile(0.99)
    lower_limit = train_data[col].quantile(0.01)
    train_data[col] = np.clip(train_data[col], lower_limit, upper_limit)

## Encode Categorical Variables

In [9]:
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    train_data[col] = le.fit_transform(train_data[col])
    label_encoders[col] = le  # Save encoder for future use

## Create Time-Based Features

In [10]:
train_data['transaction_date'] = pd.to_datetime(train_data['transaction_date'])
train_data['year'] = train_data['transaction_date'].dt.year
train_data['month'] = train_data['transaction_date'].dt.month
train_data['day'] = train_data['transaction_date'].dt.day

## Scaling numerical features

In [11]:
print("Scaling Numerical Features...")
scaler = StandardScaler()
train_data[numerical_cols] = scaler.fit_transform(train_data[numerical_cols])
print("Numerical features scaled.")

Scaling Numerical Features...
Numerical features scaled.


# Feature Engineering

## Creating Lagged features

In [12]:
train_data['lagged_account_bal'] = train_data['account_bal'].shift(1).fillna(0)
train_data['lagged_installment'] = train_data['installment'].shift(1).fillna(0)

## Creating Rolling Statistics

In [13]:
train_data['rolling_mean_account_bal'] = train_data['account_bal'].rolling(window=3).mean().fillna(train_data['account_bal'].mean())
train_data['rolling_std_account_bal'] = train_data['account_bal'].rolling(window=3).std().fillna(train_data['account_bal'].std())

## Creating Interaction Features

In [14]:
train_data['cibil_int_rate'] = train_data['cibil_score'] * train_data['int_rate']
train_data['income_loan_ratio'] = train_data['annual_inc'] / (train_data['loan_amnt'] + 1)

## Bining Numerical Features

In [15]:
train_data['cibil_score_bin'] = pd.cut(train_data['cibil_score'], bins=[0, 600, 700, 800, 900], labels=['Poor', 'Fair', 'Good', 'Excellent'])
train_data['int_rate_bin'] = pd.cut(train_data['int_rate'], bins=5, labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])

## Target based Encoding

In [16]:
for col in categorical_cols:
    target_mean = train_data.groupby(col)['loan_status'].mean()
    train_data[f'{col}_target_enc'] = train_data[col].map(target_mean)

## Create Time-Since Features

In [17]:
current_date = train_data['transaction_date'].max()
train_data['days_since_transaction'] = (current_date - train_data['transaction_date']).dt.days

## Creating Aggregated Features

In [18]:
train_data['avg_account_bal_by_grade'] = train_data.groupby('sub_grade')['account_bal'].transform('mean')
train_data['total_loans_by_purpose'] = train_data.groupby('purpose')['loan_amnt'].transform('sum')

## Save Preprocessed Data

In [19]:
train_data.to_csv('preprocessed_train_data.csv', index=False)
print("Preprocessing and Feature Engineering complete. Data saved to 'preprocessed_train_data.csv'.")

Preprocessing and Feature Engineering complete. Data saved to 'preprocessed_train_data.csv'.


In [ ]:
# Load the test dataset
test_data = pd.read_excel('test_data.xlsx')  # Replace with your test file path

print("Starting Preprocessing on Test Data...")


print("Handling Missing Values in Test Data...")
test_data = test_data.dropna(thresh=0.5 * len(test_data), axis=1)  # Drop columns with >50% missing


for col in numerical_cols:
    if col in test_data.columns:
        test_data[col] = test_data[col].fillna(train_data[col].median())


for col in categorical_cols:
    if col in test_data.columns:
        test_data[col] = test_data[col].fillna(train_data[col].mode()[0])

print("Missing values handled in Test Data.")


print("Treating Outliers in Test Data...")
for col in numerical_cols:
    if col in test_data.columns:
        upper_limit = train_data[col].quantile(0.99)
        lower_limit = train_data[col].quantile(0.01)
        test_data[col] = np.clip(test_data[col], lower_limit, upper_limit)
print("Outliers treated in Test Data.")


print("Encoding Categorical Variables in Test Data...")
for col, le in label_encoders.items():
    if col in test_data.columns:
        test_data[col] = le.transform(test_data[col])

print("Categorical variables encoded in Test Data.")


print("Creating Time-Based Features in Test Data...")
test_data['transaction_date'] = pd.to_datetime(test_data['transaction_date'])
test_data['year'] = test_data['transaction_date'].dt.year
test_data['month'] = test_data['transaction_date'].dt.month
test_data['day'] = test_data['transaction_date'].dt.day
print("Time-based features created in Test Data.")


print("Scaling Numerical Features in Test Data...")
test_data[numerical_cols] = scaler.transform(test_data[numerical_cols])
print("Numerical features scaled in Test Data.")


print("Starting Feature Engineering on Test Data...")


test_data['lagged_account_bal'] = test_data['account_bal'].shift(1).fillna(0)
test_data['lagged_installment'] = test_data['installment'].shift(1).fillna(0)


test_data['rolling_mean_account_bal'] = test_data['account_bal'].rolling(window=3).mean().fillna(test_data['account_bal'].mean())
test_data['rolling_std_account_bal'] = test_data['account_bal'].rolling(window=3).std().fillna(test_data['account_bal'].std())


test_data['cibil_int_rate'] = test_data['cibil_score'] * test_data['int_rate']
test_data['income_loan_ratio'] = test_data['annual_inc'] / (test_data['loan_amnt'] + 1)


test_data['cibil_score_bin'] = pd.cut(test_data['cibil_score'], bins=[0, 600, 700, 800, 900], labels=['Poor', 'Fair', 'Good', 'Excellent'])
test_data['int_rate_bin'] = pd.cut(test_data['int_rate'], bins=5, labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])


for col in categorical_cols:
    if col in test_data.columns:
        target_mean = train_data.groupby(col)['loan_status'].mean()
        test_data[f'{col}_target_enc'] = test_data[col].map(target_mean)


current_date = train_data['transaction_date'].max()
test_data['days_since_transaction'] = (current_date - test_data['transaction_date']).dt.days


if 'sub_grade' in test_data.columns:
    test_data['avg_account_bal_by_grade'] = test_data.groupby('sub_grade')['account_bal'].transform('mean')
if 'purpose' in test_data.columns:
    test_data['total_loans_by_purpose'] = test_data.groupby('purpose')['loan_amnt'].transform('sum')

print("Feature Engineering complete on Test Data.")

# Save Preprocessed Test Data
test_data.to_csv('preprocessed_test_data.csv', index=False)
print("Preprocessing and Feature Engineering on Test Data complete. Data saved to 'preprocessed_test_data.csv'.")


Starting Preprocessing on Test Data...
Handling Missing Values in Test Data...
Missing values handled in Test Data.
Treating Outliers in Test Data...
Outliers treated in Test Data.
Encoding Categorical Variables in Test Data...
Categorical variables encoded in Test Data.
Creating Time-Based Features in Test Data...
Time-based features created in Test Data.
Scaling Numerical Features in Test Data...
Numerical features scaled in Test Data.
Starting Feature Engineering on Test Data...
Feature Engineering complete on Test Data.
Preprocessing and Feature Engineering on Test Data complete. Data saved to 'preprocessed_test_data.csv'.
